In [1]:
import sys
import os
import requests
import datetime
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))

import time
import talib
import sqlite3 as sqlite3
import yfinance as yf
import pandas as pd
from util.util import get_ucodes

path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')

In [2]:
conn = sqlite3.connect(path_sqlite)
cursor = conn.cursor()

ucodes = get_ucodes()
data = {}
ucodes1 = ['2800.HK', '2828.HK', '2823.HK', 'spy', 'vti', 'qqq', 'shy', 'tlt'] # '3188.HK', 'voo', 'tqqq'
ucodes2 = ['hsi', 'hsce', 'ssec', 'csi300', 'sp500', 'ixic', 'tlt']
for ucode in ucodes2:
    if '.HK' in ucode:
        ucode = ucode.replace('.HK', '').zfill(5)
    
    #
    sql = """SELECT t.code, t.lot, t.nmll, t.stime, t.high, t.low, t.open, t.close, t.volume
                FROM (SELECT n.code, n.lot, n.nmll, c.stime, c.high, c.low, c.open, c.close, c.volume 
                    FROM s_{} AS c INNER JOIN name AS n 
                        ON c.code=n.code ORDER BY c.stime DESC) AS t 
                            ORDER BY t.stime """.format(ucode)
    cursor.execute(sql)
    columns = ['code', 'lot', 'nmll', 'sdate', 'high', 'low', 'open', 'last', 'vol']
    data[ucode] = pd.DataFrame(cursor.fetchall(), columns=columns)
    data[ucode].index = pd.to_datetime(data[ucode].sdate)
    print(f"{ucode}:   {data[ucode].index[0]} ~ {data[ucode].index[-1]}  {data[ucode].shape}")
conn.close()

hsi:   1986-12-31 00:00:00 ~ 2021-07-28 00:00:00  (8501, 9)
hsce:   1993-07-15 00:00:00 ~ 2021-07-28 00:00:00  (6864, 9)
ssec:   1997-07-02 00:00:00 ~ 2021-07-29 00:00:00  (5833, 9)
csi300:   2004-12-31 00:00:00 ~ 2021-07-29 00:00:00  (4074, 9)
sp500:   1927-12-30 00:00:00 ~ 2021-07-29 00:00:00  (19650, 9)
ixic:   1971-02-05 00:00:00 ~ 2021-07-29 00:00:00  (12365, 9)
tlt:   2002-07-30 00:00:00 ~ 2021-07-29 00:00:00  (4784, 9)


In [3]:
data_all = []
nmll = []
for k, df in data.items():
    data_all.append(df['last'])
    nmll.append(k)
df2 = pd.concat(data_all, axis=1)
df2 = df2.dropna()
df2.columns = nmll
print(f"{df2.index[0]} ~ {df2.index[-1]}  {df2.shape}")

2005-01-04 00:00:00 ~ 2021-07-28 00:00:00  (3783, 7)


In [4]:
# 相關系數
df2.corr()

,hsi,hsce,ssec,csi300,sp500,ixic,tlt
hsi,1.000000,0.713506,0.668706,0.799473,0.742838,0.697754,0.629937
hsce,0.713506,1.000000,0.797884,0.731752,0.150724,0.137504,0.144542
ssec,0.668706,0.797884,1.000000,0.918180,0.354980,0.333580,0.292813
csi300,0.799473,0.731752,0.918180,1.000000,0.633943,0.641073,0.580362
sp500,0.742838,0.150724,0.354980,0.633943,1.000000,0.986093,0.866912
ixic,0.697754,0.137504,0.333580,0.641073,0.986093,1.000000,0.875314
tlt,0.629937,0.144542,0.292813,0.580362,0.866912,0.875314,1.000000


In [5]:
# 年度
df3_1 = df2.copy(deep=True)
df3 = pd.DataFrame(columns=['Date']+df3_1.columns.to_list())
for year in range(2004, 2022, 1):
    df3_2 = df3_1[f'{year}-01-01': f'{year}-12-31'].iloc[[0, -1]].pct_change()*100
    df3.loc[df3.shape[0]+1] = [f'{year}-01-01 ~ {year}-12-31']+df3_2.iloc[-1].to_list()
print('年度標準差(風險)')
print(df3.std())
print('年度收益率(回報)')
df3

IndexError: positional indexers are out-of-bounds

In [ ]:
# 半年度
df3_1 = df2.copy(deep=True)
df3 = pd.DataFrame(columns=['Date']+df3_1.columns.to_list())
for year in range(2004, 2022, 1):
    #
    df3_2 = df3_1[f'{year}-01-01': f'{year}-06-30'].iloc[[0, -1]].pct_change()*100
    df3.loc[df3.shape[0]+1] = [f'{year}-01-01 ~ {year}-06-30']+df3_2.iloc[-1].to_list()
    #
    df3_3 = df3_1[f'{year}-07-01': f'{year}-12-31'].iloc[[0, -1]].pct_change()*100
    df3.loc[df3.shape[0]+1] = [f'{year}-07-01 ~ {year}-12-31']+df3_3.iloc[-1].to_list()
print('半年度標準差(風險)')
print(df3.std())
print('半年度收益率(回報)')
df3